### Report shortest shortest path out of 3 graphs with possibly negative cycles

In [35]:
## Get graphs into notebook
with open('Downloads/algo2_g1.txt') as f:
    graph1 = f.readlines()
with open('Downloads/algo2_g2.txt') as f:
    graph2 = f.readlines()
with open('Downloads/algo2_g3.txt') as f:
    graph3 = f.readlines()

In [36]:
print(graph1[:3])
print(graph2[:3])
print(graph3[:3])

['1000 47978\n', '1 14 6\n', '1 25 47\n']
['1000 47978\n', '1 2 2\n', '1 26 28\n']
['1000 47978\n', '1 8 36\n', '1 33 29\n']


So all graphs have same density: 1000 nodes and ~48K directed edges with weights.


In [37]:
## Make adjacency list representations of all graphs, with tails as keys, and sets of tuples
###                                                  (heads,  edge weights) as values

def graphDict(graphlist):
    detes = graphlist[0].strip('\n').split()
    n, m = int(detes[0]), int(detes[1])
    gDict = dict()
    for edge in graphlist[1:]:
        edge = edge.strip('\n').split()
        if int(edge[0]) in gDict:
            gDict[int(edge[0])].add((int(edge[1]), int(edge[2])))
        else:
            gDict[int(edge[0])] = {(int(edge[1]), int(edge[2]))}
    return gDict

In [38]:
gd1 = graphDict(graph1)
gd2 = graphDict(graph2)
gd3 = graphDict(graph3)

In [39]:
print(len(gd1), len(gd2), len(gd3))

1000 1000 1000


In [40]:
gd1[1]

{(14, 6),
 (25, 47),
 (70, 22),
 (82, 31),
 (98, 17),
 (134, 39),
 (146, 7),
 (189, 44),
 (192, 20),
 (261, 38),
 (283, 30),
 (340, 21),
 (380, 37),
 (381, 24),
 (403, 3),
 (422, 40),
 (423, 40),
 (425, 13),
 (518, 23),
 (520, 38),
 (547, 5),
 (558, 24),
 (578, 32),
 (601, 42),
 (604, 37),
 (640, 38),
 (648, 18),
 (699, 48),
 (707, 10),
 (712, 9),
 (723, 47),
 (729, 26),
 (730, 18),
 (769, 49),
 (793, 32),
 (830, 15),
 (841, 50),
 (852, 16),
 (874, 39),
 (880, 9),
 (890, 6),
 (897, 23),
 (917, 19),
 (949, 22),
 (988, 43),
 (993, 48)}

In [41]:
keyset = {k for k in gd1}
print(max(keyset), min(keyset))

1000 1


So they were nice enough to label the nodes consecutively from 1 to 1000.  That may help.

### Start with Bellman-Ford, since will need it for Johnson anyways

In [42]:
def bellmanFord(graphDict, source):
    '''
    Input is in adjacency form, a dictionary of tail nodes with their corresponding
    heads and edge weights as tuples in sets.
    
    Assuming also here that nodes are labeled consecutively from 1 to n.
    
    Output is the list of shortest paths from the source to each node.    
    If there are negative cycles, this function will instead output that fact.
    '''
    ## make a reverse dict with heads as keys and sets of (tail, weight) as values
    revDict = {}
    for tail in graphDict:
        for (head, weight) in graphDict[tail]:
            if head in revDict:
                revDict[head].add((tail, weight))
            else:
                revDict[head] = {(tail, weight)}
                
    ## init a list of lists for the distance matrix, and loop through nodes and i-values            
    n = max(graphDict.keys())
    L = [[float('inf') for _ in range(n+1)]]  ## start with every node unreachable using 0 (L[0]) nodes along a path
    L[0][source] = 0   ## make source reachable using 0 nodes along a shortest path
   
    for i in range(1,n):   ## never will need more than n-1 nodes along a shortest path unless negative cycles
        cutoff = True  ## terminate early if this flag doesn't change during this iteration
        shortest = [float('inf')]    ## build the new row starting with a non-existent '0' node for easier indexing
        for v in range(1,n+1):   ## find new shortest path to each v with one more path node allowed
            old = L[i-1][v]
            new = min([L[i-1][u] + dist for (u, dist) in revDict[v]])
            if new < old:
                cutoff = False
                shortest.append(new)
            else:
                shortest.append(old)
        if cutoff:
            return shortest
        L.append(shortest)
        
    ## Check for negative cycles
    for v in range(1,n+1):   ## if shortest path changes for any v now, return 'negative cycle in graph'
        old = L[i-1][v]
        new = min([L[i-1][u] + dist for (u, dist) in revDict[v]])
        if new < old:
            return 'negative cycle in graph'
    return shortest


        
    

In [7]:
paths = bellmanFord(gd1, 1)  ## test graph1 with source=1    ======> TAKES LIKE 15 SECS JUST FOR ONE SOURCE

In [8]:
if type(paths)==str:
    print(paths)

negative cycle in graph


In [9]:
paths = bellmanFord(gd2, 1)  ## test graph2 with source=1

In [10]:
if type(paths)==str:
    print(paths)

negative cycle in graph


In [43]:
paths = bellmanFord(gd3, 1)  ## test graph3 with source=1

In [44]:
if type(paths)==str:
    print(paths)

In [43]:
## OK, no neg cycles for graph3, at least reachable from source=1.  So try all sources.  Could take awhile. About 5-6 min as it turned out
result = float('inf')
for i in range(1, 1001):
    paths = bellmanFord(gd3, i)
    if type(paths)==str:
        print(paths)
        break
    shortest = min(paths)
    if shortest < result:
        result = shortest
print(result)

-19


### Since Bellman-Ford and Dijkstra's are now implemented, let's try Johnson's for time comparison

In [45]:
## modify Bellman-Ford to get Johnson weights
def johnsonWeights(graphDict):
    '''
    Input is an adjacency set dict:
    Keys are tails of directed edges, and values are sets of (head, dist) tuples.
    Assumed is a node labeling of consecutive 1 to n, for the n nodes in the graph.
    
    Output is a list of Johnson distances for each node.
    '''
    ## First add the imagined source node = n+1, with 0 distance to every other node
    fakeSource = len(graphDict)+1  ## will label it n+1, rather than 0, for easier indexing in bellman routine
    gDict = graphDict.copy()
    gDict[fakeSource] = {(v, 0) for v in gDict}
    gDict[fakeSource].add((fakeSource, 0))  ## need this self edge to make reverse dict work in bellman routine
    
    return bellmanFord(gDict, fakeSource)

In [46]:
## test on gd3
jw = johnsonWeights(gd3)

In [47]:
max(jw[1:])  ## hopefully 0, after disregarding nonexistent 0 node used for index padding

0

In [48]:
min(jw)   ## turns out to be the shortest shortest path number too

-19

In [49]:
jwGraph = {tail:{(head, dist+jw[tail]-jw[head]) for (head,dist) in gd3[tail]}for tail in gd3}

In [50]:
"""
This routine takes around 45 secs for gd3
"""

winner = 0  ## there's a negative edge in the graph so one pair s-->t has less than this upper bound
for source in jwGraph:
    graphHeap, locs, attached, unattached = heapUp(jwGraph, source)
    numNodes = len(jwGraph)
    distList = [float('inf') for _ in range(numNodes + 1)] 
    distList[source] = 0
    while graphHeap:
        swap(graphHeap, 0, -1, locs)
        newEdge = graphHeap.pop()
        distList[newEdge[0]] = newEdge[1]
        attached.add(newEdge[0])
        unattached.remove(newEdge[0])
        bubbleDown(graphHeap, locList=locs)   # since the previous last element was swapped into the removed min node's spot to maintain shape
        ## Now rescore any new crossing edges and bubble them up, since scores can only go down
        for otherEnd in jwGraph[newEdge[0]]:
            if otherEnd[0] in unattached:
                if newEdge[1] + otherEnd[1] < graphHeap[locs[otherEnd[0]]][1]:
                    graphHeap[locs[otherEnd[0]]] = (otherEnd[0], newEdge[1] + otherEnd[1])
                    bubbleUp(graphHeap, newIndex=locs[otherEnd[0]], locList=locs)


    ## Back out the johnson weights for each destination vertex in the distList
    distList = [distList[dest]+jw[dest]-jw[source] for dest, _ in enumerate(distList)]
    shortestPath = min(distList[1:])
    if shortestPath < winner:
        winner = shortestPath

In [51]:
winner

-19

## Now how about that large graph--

In [44]:
with open('Downloads/algo2_largegraph.txt') as f:
    graphlist = f.readlines()

In [45]:
graphlist[0]

'20000 999387\n'

In [70]:
gdBig = graphDict(graphlist)

Try Johnson weights for an all pairs shortest path attempt

In [71]:
jw = johnsonWeights(gdBig)

In [72]:
min(jw)   ## this actually appears to be the answer

-6

Now, pretending we don't know that's the answer, let's reweight all the graphDict edges with those jw's.

In [74]:
jwGraph = {tail:{(head, dist+jw[tail]-jw[head]) for (head,dist) in gdBig[tail]}for tail in gdBig}

### Heap 'class':

In [18]:
## First, a way to compare heap nodes by "greedy dijkstra" key.
##  Seems like heap items will be in the form of (node, score). 
def shorter(edge1, edge2):
    return edge1[-1] <= edge2[-1]

## Next, a utility to swap in place two array items while maintaining a list of their locations in the array
def swap(arr, i, j, locator=None):
    if locator:
        locator[arr[i][0]] = j
        locator[arr[j][0]] = i
    temp = arr[i]
    arr[i] = arr[j]
    arr[j] = temp
    
def bubbleUp(array, newIndex, locList=None):
    while newIndex > 0:
        oldIndex = (newIndex + 1) // 2 - 1
        if shorter(array[oldIndex], array[newIndex]):
            return
        swap(array, newIndex, oldIndex, locList)
        newIndex = oldIndex
        
def bubbleDown(array, newIndex=0, locList=None):
    # default newIndex to 0 for when the item to bubble down has just been swapped from end of array to start
    leftChild = (newIndex + 1) * 2 - 1 # left child of newIndex
    while leftChild < len(array):
        minChild = leftChild + 1  # right child of newIndex
        if minChild == len(array):  # rare case where the bubbleDown has reached a final, left child without sibling
            if shorter(array[newIndex], array[leftChild]): return
            else:
                swap(array, leftChild, newIndex, locList)
                return
        if shorter(array[leftChild], array[minChild]):
            minChild = leftChild
        if shorter(array[newIndex], array[minChild]):
            return
        swap(array, newIndex, minChild, locList)
        newIndex = minChild
        leftChild = (newIndex + 1) * 2 - 1

Run Dijkstra's on each node as source, with johnson weights

In [19]:
def heapUp(graph, source): 
    ## Build the heap
    attached = {source}
    unattached = set(graph.keys()) - attached
    heap = []
    for edge in graph[source]:
        i = len(heap)
        heap.append(edge)
        bubbleUp(heap, i)
    for node in set(graph.keys()) - {t[0] for t in graph[source]} - attached:
        heap.append((node, float('inf')))
    ## Keep track of heap locations    
    locs = [-1 for _ in range(len(graph)+1)]
    for i, v in enumerate(heap):
        locs[v[0]] = i
    return heap, locs, attached, unattached 

In [122]:
"""
This routine takes a little over 1 second per source, so for the 20K node graph, maybe 6 hours.
"""

winner = 0  ## there's a negative edge in the graph so one pair s-->t has less than this upper bound
for source in gdBig:
    graphHeap, locs, attached, unattached = heapUp(jwGraph, source)
    numNodes = len(gdBig)
    distList = [float('inf') for _ in range(numNodes + 1)] 
    distList[source] = 0
    while graphHeap:
        swap(graphHeap, 0, -1, locs)
        newEdge = graphHeap.pop()
        distList[newEdge[0]] = newEdge[1]
        attached.add(newEdge[0])
        unattached.remove(newEdge[0])
        bubbleDown(graphHeap, locList=locs)   # since the previous last element was swapped into the removed min node's spot to maintain shape
        ## Now rescore any new crossing edges and bubble them up, since scores can only go down
        for otherEnd in gdBig[newEdge[0]]:
            if otherEnd[0] in unattached:
                if newEdge[1] + otherEnd[1] < graphHeap[locs[otherEnd[0]]][1]:
                    graphHeap[locs[otherEnd[0]]] = (otherEnd[0], newEdge[1] + otherEnd[1])
                    bubbleUp(graphHeap, newIndex=locs[otherEnd[0]], locList=locs)


    ## Back out the johnson weights for each destination vertex in the distList
    distList = [distList[dest]+jw[source]-jw[dest] for dest, _ in enumerate(distList)]
    shortestPath = min(distList[1:])
    if shortestPath < winner:
        winner = shortestPath

In [123]:
winner   ## well at least it works

-6

### Now can Floyd-Warshall solve it faster?  Maybe the smaller graphs but O(n^3) isn't going to fit 20K node graph into this computer

Implement F-W just for practice:

In [125]:
def fw(graphDict):
    ## run the dynamic routine for shortest paths using internal hops thru nodes 1-k for incrementally larger k's
    n = len(graphDict)
    oldK = [[float('inf') for s in range(n+1)] for t in range(n+1)]
    for s in range(1,n+1):
        oldK[s][s] = 0  ## all nodes reachable from self using k=0 intermediary nodes
    for s in graphDict:
        for (t,dist) in graphDict[s]:
            oldK[s][t] = dist   ## all heads reachable from all tails with C(tail,head) cost using k=0 intermediary nodes
    for k in range(1,n+1):
        oldK = [[min(oldK[s][t], oldK[s][k] + oldK[k][t]) for t in range(n+1)] for s in range(n+1)]
        
    return oldK

In [124]:
sp3 = fw(graphDict(graph3))   ### takes like 9 min---blah
for s in range(len(sp3)):
    if sp3[s][s] < 0 :    ## check for negative cycles
        print(sp3[s][s])
min([min(path) for path in sp3]) 

-19

In [ ]:
bigFW = fw(gdBig)   ## TOO LONGGGGGGGG

## Johnson's 1000 Dijkstras plus 1 Bellman-Ford take 45 seconds for gd3.  Vs. 6 minutes for 1000 B-F's, or 9 minutes for a Floyd-Warshall.  